# Droyßen Authorship-Attribution

## Importe

In [1]:
import os

import sklearn.feature_extraction.text as text
import sklearn.preprocessing as preprocessing

import sklearn.model_selection as model_selection
import sklearn.metrics as metrics

import scipy.spatial.distance as scidist
import sklearn.decomposition
import scipy.cluster.hierarchy as hierarchy

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Preprocessing

### Helferfunktion zum Laden der Daten

In [2]:
def load_directory(directory, max_length):
    documents, authors, titles = [], [], [] 
    for filename in os.scandir(directory):
        if not filename.name.endswith('.txt'):
            continue
        author, _ = os.path.splitext(filename.name)

        with open(filename.path) as f:
            contents = f.read()
        lemmas = contents.lower().split()
        start_idx, end_idx, segm_cnt = 0, max_length, 1

        # extract slices from the text:
        while end_idx < len(lemmas):
            documents.append(' '.join(lemmas[start_idx:end_idx]))
            authors.append(author[0])
            title = filename.name.replace('.txt', '').split('_')[1]
            titles.append(f"{title}-{segm_cnt}")

            start_idx += max_length
            end_idx += max_length
            segm_cnt += 1

    return documents, authors, titles

Vokabular: verschiedene Listen: Ausgangspunkt: most frequent words, dann schrittweise neue Listen erstellen mit Wörtern, die man rausnimmt (mit Begründung im Text), dann über Code von Ursprungsliste "abziehen"
Anders als Kestemon: dort war die Liste mit "##" aussortiert